# Documentación de la Extracción de Datos Climáticos 🧑‍💻

En esta sección, detallaremos el proceso que seguimos para obtener los datos de temperatura máxima y mínima de Pinar del Río, Cuba, utilizando la API de Open-Meteo. Este script fue diseñado para preparar nuestro dataset de series de tiempo.

## 1. Configuración del Entorno e Importación de Librerías
Lo primero que hice fue importar las bibliotecas de Python necesarias para realizar nuestra tarea:

In [1]:
import pandas as pd
import requests

* pandas: La importé para manejar y estructurar los datos extraídos en un formato tabular (DataFrame), lo cual es esencial para el análisis posterior y la exportación a CSV.

* requests: La usé para realizar la solicitud HTTP a la API de Open-Meteo, que es la forma en que "pedimos" los datos desde Internet.

## 2. Definición de Parámetros de la API
A continuación, establecí todas las variables que definen qué datos queremos obtener.

In [ ]:
# 1. Definir los parámetros de la API Open-Meteo, usando Pinar del Río
LATITUD = 22.4167       # Latitud de Pinar del Río, Cuba
LONGITUD = -83.6961     # Longitud de Pinar del Río, Cuba
FECHA_INICIO = "2024-01-01"
FECHA_FIN = "2025-11-01"
VARIABLES = "temperature_2m_max,temperature_2m_min"
ZONA_HORARIA = "America/Havana"
URL_BASE = "https://archive-api.open-meteo.com/v1/archive"

* Coordenadas (LATITUD/LONGITUD): Fijamos las coordenadas geográficas que identifican la ubicación de Pinar del Río.

* Rango de Fechas: Establecimos la ventana de tiempo exacta para la serie: desde el 1 de enero de 2024 hasta el 1 de noviembre de 2025. Usé la URL archive-api porque estamos pidiendo datos históricos.

* VARIABLES: Indicamos explícitamente a la API que solo necesitamos dos series de tiempo diarias: temperature_2m_max (Máxima) y temperature_2m_min (Mínima).

## 3. Solicitud de Datos a la API
Aquí es donde construí la solicitud y la envié a los servidores de Open-Meteo, utilizando un bloque try...except para manejar cualquier error de conexión.

In [ ]:
# 2. Construir la URL completa para la solicitud
parametros = {
    # ... diccionario con latitud, longitud, etc. ...
}

# 3. Realizar la solicitud HTTP y obtener la respuesta JSON
try:
    respuesta = requests.get(URL_BASE, params=parametros)
    respuesta.raise_for_status()  
    datos = respuesta.json()
    # ...
except requests.exceptions.RequestException as e:
    print(f"❌ Error al conectar con la API: {e}")

1. Ensamblaje de Parámetros: Primero, organicé todos mis parámetros definidos en un diccionario llamado parametros.

2. Llamada GET: Luego, utilicé requests.get() para enviar la solicitud HTTP, adjuntando el diccionario de parámetros a la URL_BASE.

3. Manejo de Errores: La línea respuesta.raise_for_status() es crucial; si la API devuelve un error (por ejemplo, un error 404 o 500), esta línea lo detecta y lo lanza como una excepción, permitiendo que mi bloque except me informe del problema.

4. Decodificación: Si la solicitud fue exitosa, utilicé respuesta.json() para convertir el texto plano de la respuesta en un objeto de Python (datos), que normalmente es un diccionario.

## 4. Procesamiento de Datos y Creación del DataFrame
En este paso, transformé el diccionario datos (la respuesta JSON) en nuestro objeto DataFrame de Pandas, listo para el análisis.

In [ ]:
# 4. Procesar los datos y crear un DataFrame de Pandas
if 'daily' in datos:
    # Extraer las series de tiempo del JSON
    fechas = datos['daily']['time']
    temp_max = datos['daily']['temperature_2m_max']
    temp_min = datos['daily']['temperature_2m_min']

    # Crear el DataFrame
    df_clima_pr = pd.DataFrame({
        'Fecha': fechas,
        'Temp_Maxima_C': temp_max,
        'Temp_Minima_C': temp_min
    })

    # Convertir la columna de Fecha al formato datetime
    df_clima_pr['Fecha'] = pd.to_datetime(df_clima_pr['Fecha'])
    df_clima_pr = df_clima_pr.set_index('Fecha')
    # ...

1. Extracción de Listas: Navegué dentro del diccionario datos hasta la clave 'daily' (donde la API almacena los datos diarios) y extraje las listas de time, temperature_2m_max y temperature_2m_min.

2. Creación del DataFrame: Utilicé estas listas para crear un DataFrame con las columnas Fecha, Temp_Maxima_C y Temp_Minima_C.

3. Formato de Fecha: Transformé la columna 'Fecha' (que originalmente era una cadena de texto, string) al tipo de dato datetime usando pd.to_datetime().

4. Índice de Serie de Tiempo: Finalmente, establecí la columna 'Fecha' como el índice del DataFrame usando .set_index('Fecha'). Esto es fundamental para trabajar con series de tiempo en Pandas.

##  5. Exportación a CSV

El último paso fue guardar nuestro dataset limpio y estructurado.

In [ ]:
# 5. Exportar el DataFrame a un archivo CSV
nombre_archivo = "pinar_del_rio_temperaturas_2024_2025.csv"
df_clima_pr.to_csv(nombre_archivo)

print("✅ Extracción y guardado exitoso.")
# ...

Usé el método .to_csv() para exportar el DataFrame completo a un archivo llamado pinar_del_rio_temperaturas_2024_2025.csv, que es el que utilizaremos para todos nuestros análisis de modelos.